In [1]:
import pandas as pd
import numpy as np

import os
import sys
sys.path.append('..')

from pathlib import Path
import datetime as dt

import wget
import requests
from zipfile import ZipFile

## Data Collection

Coletando dados de ITR, DFP e FRE

In [2]:
def download_cvm_data(
    data_type,
    initial_year=2011, 
    last_year=2022, 
    save_folder='.'
):

    download_path = os.path.join(save_folder, 'downloaded')
    os.makedirs(download_path, exist_ok=True)

    if data_type=='fre':
        BASE_URL = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    elif data_type=='itr':
        BASE_URL = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    if data_type=='dfp':
        BASE_URL = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

    filename_list = []
    for year in range(initial_year, last_year + 1):
        filename_list.append(f'{data_type}_cia_aberta_{year}.zip')


    for filename in filename_list:
        try:
            save_path = os.path.join(download_path, filename)
            
            if not Path(save_path).exists():
                wget.download(
                    BASE_URL + filename, 
                    out=save_path
                )
                
        except Exception as e:
            print(f'The file of year {year} was not found.')
            print('Exception Message:', e, end='\n\n')


In [3]:
START_YEAR = 2011
LAST_YEAR = 2022

itr_save_folder = os.path.join('..', 'cvm_data', 'itr')
fre_save_folder = os.path.join('..', 'cvm_data', 'fre')
dfp_save_folder = os.path.join('..', 'cvm_data', 'dfp')

download_cvm_data('itr', START_YEAR, LAST_YEAR, save_folder=itr_save_folder)
download_cvm_data('dfp', START_YEAR, LAST_YEAR, save_folder=dfp_save_folder)
download_cvm_data('fre', START_YEAR, LAST_YEAR, save_folder=fre_save_folder)

In [632]:
def extract_all_files(folder_path):
    filename_list = os.listdir(folder_path)

    extract_path = os.path.join(Path(folder_path).parent, 'extracted')
    for filename in filename_list:
        ZipFile(
            os.path.join(folder_path, filename), 
            'r'
        ).extractall(extract_path)


extract_all_files(os.path.join(itr_save_folder, 'downloaded'))
extract_all_files(os.path.join(fre_save_folder, 'downloaded'))
extract_all_files(os.path.join(dfp_save_folder, 'downloaded'))

Consolidando os dados anuais

In [633]:
def consolidate_years(folder_path, data_type, document_names, initial_year, last_year):

    for document_name in document_names:
        df_file = pd.DataFrame()
        for year in range(initial_year, last_year+1):
            df_file = pd.concat([
                df_file,
                pd.read_csv(
                    os.path.join(
                        folder_path,
                        f'{data_type}_cia_aberta_{document_name}_{year}.csv'
                    ),
                    sep=';',
                    decimal=',',
                    encoding='ISO-8859-1'
                )
            ])

        save_path = os.path.join(
            Path(folder_path).parent, 
            'consolidated',
            f'{data_type}_{document_name}_2011_2021.csv'
        )
        os.makedirs(Path(save_path).parent, exist_ok=True)

        df_file.to_csv(save_path)


document_names = ['DRE_con', 'BPA_con', 'BPP_con', 'DFC_MI_con']
consolidate_years(
    os.path.join(itr_save_folder, 'extracted'),
    'itr', 
    document_names, 
    2011, 2021
)

document_names = ['DRE_con', 'BPA_con', 'BPP_con', 'DFC_MI_con']
consolidate_years(
    os.path.join(dfp_save_folder, 'extracted'),
    'dfp', 
    document_names, 
    2011, 2021
)

document_names = ['capital_social']
consolidate_years(
    os.path.join(fre_save_folder, 'extracted'),
    'fre', 
    document_names, 
    2011, 2021
)

## Data Processing

In [3]:
df_ = pd.read_csv(os.path.join('..', 'cvm_data', 'itr', 'consolidated', 'itr_DRE_con_2011_2021.csv'))
df_[df_['DENOM_CIA'].str.contains('ITAU')].drop_duplicates('DENOM_CIA')

,Unnamed: 0,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
36172,36172,07.256.507/0001-50,2011-03-31,1,ITAUSEG PARTICIPAÇÕES SA,19933,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receita de Venda de Bens e/ou Serviços,1128601.0,S
88844,88844,54.526.082/0001-31,2011-03-31,2,ITAUTEC S.A. - GRUPO ITAUTEC,12530,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receita de Venda de Bens e/ou Serviços,348476.0,S
95088,95088,60.872.504/0001-23,2011-03-31,1,ITAU UNIBANCO HOLDING S.A.,19348,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receitas da Intermediação Financeira,18190098.0,S


In [4]:
COD_CVM = 19348
CNPJ = '60.872.504/0001-23'
TICKER_NAME = 'ITUB4.SA'

In [5]:
# Funções de preprocessamento

def preprocess_dre(codigo_cvm):

    df_dfp = pd.read_csv('..\cvm_data\dfp\consolidated\dfp_DRE_con_2011_2021.csv', index_col=0)
    df_dfp.insert(11, 'TIPO_PERIODO','ANUAL')
    df_itr = pd.read_csv('..\cvm_data\itr\consolidated\itr_DRE_con_2011_2021.csv', index_col=0)
    df_itr.insert(11, 'TIPO_PERIODO','TRIMESTRAL')

    df_dre = pd.concat([df_dfp, df_itr])

    df_dre = df_dre[df_dre['CD_CVM'] == codigo_cvm]
    df_dre = df_dre[df_dre['ORDEM_EXERC'] == 'ÚLTIMO']

    df_dre['DT_REFER'] = pd.to_datetime(df_dre['DT_REFER'])
    df_dre['DT_INI_EXERC'] = pd.to_datetime(df_dre['DT_INI_EXERC'])
    df_dre['DT_FIM_EXERC'] = pd.to_datetime(df_dre['DT_FIM_EXERC'])

    # IGNORA SEMESTRAL
    period_days = (df_dre['DT_FIM_EXERC'] - df_dre['DT_INI_EXERC']).dt.days
    df_dre = df_dre[(period_days<100) | (period_days>360)]

    return df_dre

def preprocess_bpp(codigo_cvm):

    df_dfp = pd.read_csv('..\cvm_data\dfp\consolidated\dfp_BPP_con_2011_2021.csv', index_col=0)
    df_itr = pd.read_csv('..\cvm_data\itr\consolidated\itr_BPP_con_2011_2021.csv', index_col=0)

    df_bpp = pd.concat([df_dfp, df_itr])

    df_bpp = df_bpp[df_bpp['CD_CVM'] == COD_CVM]
    df_bpp = df_bpp[df_bpp['ORDEM_EXERC'] == 'ÚLTIMO']

    df_bpp['DT_REFER'] = pd.to_datetime(df_bpp['DT_REFER'])
    df_bpp['DT_FIM_EXERC'] = pd.to_datetime(df_bpp['DT_FIM_EXERC'])

    return df_bpp


def get_dfc_mi(cod_cvm):
    df_dfp = pd.read_csv('..\cvm_data\dfp\consolidated\dfp_DFC_MI_con_2011_2021.csv', index_col=0)
    df_dfp.insert(11, 'TIPO_PERIODO','ANUAL')
    df_itr = pd.read_csv('..\cvm_data\itr\consolidated\itr_DFC_MI_con_2011_2021.csv', index_col=0)
    df_itr.insert(11, 'TIPO_PERIODO','TRIMESTRAL')

    df_dfc = pd.concat([df_dfp, df_itr])

    df_dfc = df_dfc[df_dfc['CD_CVM'] == cod_cvm]
    df_dfc = df_dfc[df_dfc['ORDEM_EXERC'] == 'ÚLTIMO']

    df_dfc['DT_REFER'] = pd.to_datetime(df_dfc['DT_REFER'])
    df_dfc['DT_INI_EXERC'] = pd.to_datetime(df_dfc['DT_INI_EXERC'])
    df_dfc['DT_FIM_EXERC'] = pd.to_datetime(df_dfc['DT_FIM_EXERC'])

    return df_dfc

def preprocess_bpa(codigo_cvm):

    df_dfp = pd.read_csv('..\cvm_data\dfp\consolidated\dfp_BPA_con_2011_2021.csv', index_col=0)
    df_itr = pd.read_csv('..\cvm_data\itr\consolidated\itr_BPA_con_2011_2021.csv', index_col=0)

    df = pd.concat([df_dfp, df_itr])

    df = df[df['CD_CVM'] == COD_CVM]
    df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']

    df['DT_REFER'] = pd.to_datetime(df['DT_REFER'])
    df['DT_FIM_EXERC'] = pd.to_datetime(df['DT_FIM_EXERC'])

    return df

def preprocess_fre(codigo_cvm):
    df_fre = pd.read_csv('../cvm_data/fre/consolidated/fre_capital_social_2011_2021.csv', index_col=0)
    df_fre['Data_Referencia'] = pd.to_datetime(df_fre['Data_Referencia'])
    df_fre = df_fre[df_fre['CNPJ_Companhia'] == CNPJ]
    
    return df_fre


In [6]:
df_dre = preprocess_dre(COD_CVM)
df_bpa = preprocess_bpa(COD_CVM)
df_dfc = get_dfc_mi(COD_CVM)
df_bpp = preprocess_bpp(COD_CVM)
df_fre = preprocess_fre(COD_CVM)

In [21]:
# Funções para coleta de dados dos balanços

def get_lucro_liquido_trimestral(df_dre):
    df_dre = df_dre.copy()
    
    first_year = df_dre['DT_REFER'].dt.year.min()
    last_year = df_dre['DT_REFER'].dt.year.max()

    df_dre = df_dre.loc[
        (df_dre['DS_CONTA'].str.lower().str.contains('lucro')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('preju')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('consolidado'))
    ].copy()
    for year in range(first_year, last_year+1):

        df_dre_temp = df_dre[(df_dre['DT_REFER'].dt.year == year)]

        trimestral_sum = df_dre_temp[df_dre_temp['TIPO_PERIODO']=='TRIMESTRAL']['VL_CONTA'].sum()

        anual_index = df_dre_temp[df_dre_temp['TIPO_PERIODO']=='ANUAL'].index[0]

        df_dre.loc[anual_index, 'VL_CONTA'] -= trimestral_sum
        df_dre.loc[anual_index, 'TIPO_PERIODO'] = 'TRIMESTRAL'

    df_dre = df_dre[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
    df_dre.columns = ['cnpj', 'dt_refer', 'value']
    df_dre.insert(2, 'description', 'lucro_liquido')

    return df_dre

def get_qtd_acoes(df_fre):
    df_fre = df_fre.copy()
    df_fre = df_fre[df_fre['Tipo_Capital'] == 'Capital Integralizado']
    df_fre = df_fre[['CNPJ_Companhia', 'Data_Referencia', 'Quantidade_Total_Acoes']]
    df_fre.columns = ['cnpj', 'dt_refer', 'value']
    df_fre.insert(2, 'description', 'qtd_acoes')
    return df_fre


def get_patrimonio_liquido(df_bpp):
    df_bpp = df_bpp.copy()
    df_bpp = df_bpp[
        (df_bpp['DS_CONTA'].str.lower().str.contains('patrimônio líquido'))
    ]
    df_bpp = df_bpp[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
    df_bpp.columns = ['cnpj', 'dt_refer', 'value']
    df_bpp.insert(2, 'description', 'patrimonio_liquido')
    return df_bpp


def get_divida_bruta(df_bpp):
    df_bpp = df_bpp.copy()
    df_bpp = df_bpp[
        df_bpp['DS_CONTA'].str.lower().str.contains('empr') & 
        df_bpp['DS_CONTA'].str.lower().str.contains('finan') &
        (df_bpp['CD_CONTA'].str.len() == 7)
    ].groupby(['CNPJ_CIA', 'DT_REFER']).sum()[['VL_CONTA']].reset_index()

    df_bpp.columns = ['cnpj', 'dt_refer', 'value']
    df_bpp.insert(2, 'description', 'divida_bruta')
    
    return df_bpp

def get_deprec_e_amort_trimestral(df_dfc):
    df_dfc = df_dfc.copy()
    
    first_year = df_dfc['DT_REFER'].dt.year.min()
    last_year = df_dfc['DT_REFER'].dt.year.max()

    df_dfc = df_dfc.loc[
        (df_dfc['DS_CONTA'].str.lower().str.contains('deprec')) &
        (df_dfc['DS_CONTA'].str.lower().str.contains('amort'))
    ].sort_values('DT_REFER')

    for year in range(first_year, last_year+1):

        df_dfc_temp = df_dfc[(df_dfc['DT_REFER'].dt.year == year)]

        for i, (index, row) in enumerate(df_dfc_temp.iterrows()):
            if i > 0:
                df_dfc.loc[index, 'VL_CONTA'] = df_dfc_temp.iloc[i]['VL_CONTA'] - df_dfc_temp.iloc[i-1]['VL_CONTA']

    df_dfc = df_dfc[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
    df_dfc.columns = ['cnpj', 'dt_refer', 'value']
    df_dfc.insert(2, 'description', 'deprec_e_amort')

    return df_dfc

def get_caixa(df_bpa):
    df_bpa = df_bpa.copy()
    df_bpa = df_bpa[
        (df_bpa['DS_CONTA'].str.lower().str.contains('caixa') &
        df_bpa['DS_CONTA'].str.lower().str.contains('equiv')) |
        (df_bpa['DS_CONTA'].str.lower().str.contains('aplicações financeiras') &
        (df_bpa['CD_CONTA'].str.len()==7))
    ].groupby(['CNPJ_CIA', 'DT_REFER']).sum()['VL_CONTA'].reset_index()

    df_bpa.columns = ['cnpj', 'dt_refer', 'value']
    df_bpa.insert(2, 'description', 'caixa')
    return df_bpa

def get_ebit(df_dre):
    df_dre = df_dre.copy()
        
    first_year = df_dre['DT_REFER'].dt.year.min()
    last_year = df_dre['DT_REFER'].dt.year.max()

    df_dre = df_dre.loc[
        ((df_dre['DS_CONTA'].str.lower().str.contains('resultado bruto')) |
        ((df_dre['DS_CONTA'].str.lower().str.contains('despesas')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('rece')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('opera')))) &
        (df_dre['CD_CONTA'].str.lower().str.len() == 4)
    ].copy()

    df_dre = df_dre.groupby(
        ['CNPJ_CIA', 'DT_REFER', 'TIPO_PERIODO']
    ).sum()['VL_CONTA'].reset_index()

    for year in range(first_year, last_year+1):

        df_dre_temp = df_dre[(df_dre['DT_REFER'].dt.year == year)]

        trimestral_sum = df_dre_temp[df_dre_temp['TIPO_PERIODO']=='TRIMESTRAL']['VL_CONTA'].sum()

        anual_index = df_dre_temp[df_dre_temp['TIPO_PERIODO']=='ANUAL'].index[0]

        df_dre.loc[anual_index, 'VL_CONTA'] -= trimestral_sum
        df_dre.loc[anual_index, 'TIPO_PERIODO'] = 'TRIMESTRAL'
    
    df_dre = df_dre[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
    df_dre.columns = ['cnpj', 'dt_refer', 'value']
    df_dre.insert(2, 'description', 'ebit')

    return df_dre

def get_receita_liquida(df_dre):
    df_dre = df_dre.copy()
    
    first_year = df_dre['DT_REFER'].dt.year.min()
    last_year = df_dre['DT_REFER'].dt.year.max()

    df_dre = df_dre.loc[
        ((df_dre['DS_CONTA'].str.lower().str.contains('receita')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('bens')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('serviços'))) |
        ((df_dre['DS_CONTA'].str.lower().str.contains('receita')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('intermedia')) &
        (df_dre['DS_CONTA'].str.lower().str.contains('financei')))
    ].copy()

    for year in range(first_year, last_year+1):
        df_dre_temp = df_dre[(df_dre['DT_REFER'].dt.year == year)]

        trimestral_sum = df_dre_temp[df_dre_temp['TIPO_PERIODO']=='TRIMESTRAL']['VL_CONTA'].sum()

        anual_index = df_dre_temp[df_dre_temp['TIPO_PERIODO']=='ANUAL'].index[0]

        df_dre.loc[anual_index, 'VL_CONTA'] -= trimestral_sum
        df_dre.loc[anual_index, 'TIPO_PERIODO'] = 'TRIMESTRAL'

    df_dre = df_dre[['CNPJ_CIA', 'DT_REFER', 'VL_CONTA']]
    df_dre.columns = ['cnpj', 'dt_refer', 'value']
    df_dre.insert(2, 'description', 'receita_liquida')

    return df_dre.sort_values('dt_refer')

In [22]:
df_petr_lucro_liquido = get_lucro_liquido_trimestral(df_dre)
df_qtd_acoes = get_qtd_acoes(df_fre)
df_patrimonio_liquido = get_patrimonio_liquido(df_bpp)
df_divida_bruta = get_divida_bruta(df_bpp)
df_deprec_amort = get_deprec_e_amort_trimestral(df_dfc)
df_caixa = get_caixa(df_bpa)
df_ebit = get_ebit(df_dre)
df_receita_liquida = get_receita_liquida(df_dre)




In [23]:
df = pd.concat([
    df_petr_lucro_liquido, df_qtd_acoes, df_patrimonio_liquido,
    df_divida_bruta, df_deprec_amort, df_caixa, df_ebit, df_receita_liquida
])
df.insert(0, 'ticker', TICKER_NAME)
df = df.pivot_table(
    index=['ticker', 'cnpj', 'dt_refer'],
    columns='description',
    values='value'
)

df['qtd_acoes'] = df['qtd_acoes'].fillna(method='ffill')

df = df.dropna()

df['lucro_liquido'] = df['lucro_liquido'].rolling(4).sum()
df['receita_liquida'] = df['receita_liquida'].rolling(4).sum()
df['ebit'] = df['ebit'].rolling(4).sum()
df['deprec_e_amort'] = df['deprec_e_amort'].rolling(4).sum()

df = df.dropna()


df = df.reset_index()
df

description,ticker,cnpj,dt_refer,caixa,deprec_e_amort,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida
0,ITUB4.SA,60.872.504/0001-23,2011-12-31,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0
1,ITUB4.SA,60.872.504/0001-23,2012-03-31,422560987.0,2173898.0,18475746.0,14518396.0,74388572.0,5.028030e+09,107275454.0
2,ITUB4.SA,60.872.504/0001-23,2012-12-31,504038000.0,3290238.0,25604300.0,20280231.0,75998000.0,5.028030e+09,159373059.0
3,ITUB4.SA,60.872.504/0001-23,2013-03-31,492470000.0,3302690.0,27541843.0,19772467.0,77177000.0,5.530833e+09,153493949.0
4,ITUB4.SA,60.872.504/0001-23,2013-06-30,495625000.0,3346000.0,26245000.0,20448000.0,78669000.0,5.530833e+09,152164000.0
5,ITUB4.SA,60.872.504/0001-23,2013-09-30,509308000.0,3393088.0,26885748.0,21150773.0,81300000.0,5.530833e+09,152913486.0
6,ITUB4.SA,60.872.504/0001-23,2013-12-31,513674000.0,2333000.0,20865000.0,16522000.0,84192000.0,5.530833e+09,94797000.0
7,ITUB4.SA,60.872.504/0001-23,2014-03-31,511423000.0,2386000.0,23249000.0,17633000.0,85487000.0,6.083916e+09,101563000.0
8,ITUB4.SA,60.872.504/0001-23,2014-06-30,518829000.0,2398000.0,26600000.0,18709000.0,89331000.0,6.083916e+09,106183000.0
9,ITUB4.SA,60.872.504/0001-23,2014-09-30,535793000.0,2463000.0,27329000.0,20374000.0,94805000.0,6.083916e+09,116213000.0


In [24]:
import yfinance

ticker = yfinance.Ticker(TICKER_NAME)

df_close = ticker.history(
    start=df['dt_refer'].min(),
    end=df['dt_refer'].max() + dt.timedelta(days=1),
    auto_adjust=False
).reset_index()

df_close = df_close[['Date', 'Close', 'Dividends']]
df_close.columns = ['dt_refer', 'close_price', 'dividends']
df_close

# TODO: Calculate dividend yield



,dt_refer,close_price,dividends
0,2012-01-02,15.495298,0.005464
1,2012-01-03,15.882339,0.000000
2,2012-01-04,15.900553,0.000000
3,2012-01-05,15.782164,0.000000
4,2012-01-06,15.900553,0.000000
...,...,...,...
2474,2021-12-23,21.340000,0.000000
2475,2021-12-27,21.559999,0.000000
2476,2021-12-28,21.480000,0.000000
2477,2021-12-29,21.299999,0.000000


In [25]:
df_final = df.merge(df_close, how='outer', on='dt_refer')
df_final = df_final.sort_values('dt_refer')
df_final = df_final.fillna(method='ffill')
df_final = df_final.dropna()
df_final

,ticker,cnpj,dt_refer,caixa,deprec_e_amort,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida,close_price,dividends
39,ITUB4.SA,60.872.504/0001-23,2012-01-02,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,15.495298,0.005464
40,ITUB4.SA,60.872.504/0001-23,2012-01-03,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,15.882339,0.000000
41,ITUB4.SA,60.872.504/0001-23,2012-01-04,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,15.900553,0.000000
42,ITUB4.SA,60.872.504/0001-23,2012-01-05,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,15.782164,0.000000
43,ITUB4.SA,60.872.504/0001-23,2012-01-06,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,15.900553,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,ITUB4.SA,60.872.504/0001-23,2021-12-27,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,21.559999,0.000000
2496,ITUB4.SA,60.872.504/0001-23,2021-12-28,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,21.480000,0.000000
2497,ITUB4.SA,60.872.504/0001-23,2021-12-29,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,21.299999,0.000000
2498,ITUB4.SA,60.872.504/0001-23,2021-12-30,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,20.950001,0.000000


In [26]:
df_final['dt_refer'] = df_final['dt_refer'].shift(-20) 
df_final['close_price'] = df_final['close_price'].shift(-20) 

df_final = df_final.dropna()
df_final

,ticker,cnpj,dt_refer,caixa,deprec_e_amort,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida,close_price,dividends
39,ITUB4.SA,60.872.504/0001-23,2012-01-31,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,15.991621,0.005464
40,ITUB4.SA,60.872.504/0001-23,2012-02-01,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,16.255720,0.000000
41,ITUB4.SA,60.872.504/0001-23,2012-02-02,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,16.255720,0.000000
42,ITUB4.SA,60.872.504/0001-23,2012-02-03,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,16.319468,0.000000
43,ITUB4.SA,60.872.504/0001-23,2012-02-06,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,16.419643,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2475,ITUB4.SA,60.872.504/0001-23,2021-12-27,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,21.559999,0.000000
2476,ITUB4.SA,60.872.504/0001-23,2021-12-28,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,21.480000,0.000000
2477,ITUB4.SA,60.872.504/0001-23,2021-12-29,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,21.299999,0.000000
2478,ITUB4.SA,60.872.504/0001-23,2021-12-30,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,20.950001,0.017650


In [28]:
df_final['EBITDA'] = df_final['ebit'] + df_final['deprec_e_amort']
# df_final['divida_liquida'] = df_final['divida_bruta'] - df_final['caixa']
df_final['LPA'] = df_final['lucro_liquido']*1000 / df_final['qtd_acoes']
df_final['VPA'] = df_final['patrimonio_liquido']*1000 / df_final['qtd_acoes']


df_final['P/L'] = df_final['close_price'] / df_final['LPA']
df_final['P/EBITDA'] = df_final['close_price'] / df_final['EBITDA']
df_final['P/VPA'] = df_final['close_price'] / df_final['VPA']
# df_final['DL/PL'] = df_final['divida_liquida'] / df_final['patrimonio_liquido']
# df_final['DL/EBITDA'] = df_final['divida_liquida'] / df_final['EBITDA']
# df_final['DL/EBIT'] = df_final['divida_liquida'] / df_final['ebit']
df_final['ROE'] = df_final['lucro_liquido'] / df_final['patrimonio_liquido']
df_final['MARGEM_EBITDA'] = df_final['EBITDA'] / df_final['receita_liquida']
df_final['MARGEM_EBIT'] = df_final['ebit'] / df_final['receita_liquida']
df_final['MARGEM_LIQUIDA'] = df_final['lucro_liquido'] / df_final['receita_liquida']

df_final

c:\ProgramData\Anaconda3\envs\rl_study\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\ProgramData\Anaconda3\envs\rl_study\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\ProgramData\Anaconda3\envs\rl_study\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to 

,ticker,cnpj,dt_refer,caixa,deprec_e_amort,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida,...,EBITDA,LPA,VPA,P/L,P/EBITDA,P/VPA,ROE,MARGEM_EBITDA,MARGEM_EBIT,MARGEM_LIQUIDA
39,ITUB4.SA,60.872.504/0001-23,2012-01-31,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,...,20419000.0,3.196282,16.481525,5.003195,7.831736e-07,0.970276,0.193931,0.197093,0.176166,0.141022
40,ITUB4.SA,60.872.504/0001-23,2012-02-01,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,...,20419000.0,3.196282,16.481525,5.085822,7.961076e-07,0.986299,0.193931,0.197093,0.176166,0.141022
41,ITUB4.SA,60.872.504/0001-23,2012-02-02,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,...,20419000.0,3.196282,16.481525,5.085822,7.961076e-07,0.986299,0.193931,0.197093,0.176166,0.141022
42,ITUB4.SA,60.872.504/0001-23,2012-02-03,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,...,20419000.0,3.196282,16.481525,5.105766,7.992295e-07,0.990167,0.193931,0.197093,0.176166,0.141022
43,ITUB4.SA,60.872.504/0001-23,2012-02-06,401480000.0,2168000.0,18251000.0,14610000.0,75336000.0,4.570936e+09,103601000.0,...,20419000.0,3.196282,16.481525,5.137108,8.041355e-07,0.996245,0.193931,0.197093,0.176166,0.141022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,ITUB4.SA,60.872.504/0001-23,2021-12-27,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,...,49683000.0,3.013422,16.213770,7.154656,4.339512e-07,1.329734,0.185856,0.254186,0.234576,0.151152
2476,ITUB4.SA,60.872.504/0001-23,2021-12-28,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,...,49683000.0,3.013422,16.213770,7.128108,4.323410e-07,1.324800,0.185856,0.254186,0.234576,0.151152
2477,ITUB4.SA,60.872.504/0001-23,2021-12-29,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,...,49683000.0,3.013422,16.213770,7.068375,4.287181e-07,1.313698,0.185856,0.254186,0.234576,0.151152
2478,ITUB4.SA,60.872.504/0001-23,2021-12-30,42222000.0,3833000.0,45850000.0,29544000.0,158962000.0,9.804135e+09,195459000.0,...,49683000.0,3.013422,16.213770,6.952229,4.216734e-07,1.292112,0.185856,0.254186,0.234576,0.151152


In [29]:
save_path = os.path.join('..', 'cvm_data', f'{TICKER_NAME}_FUND.csv')
df_final_to_save = df_final.rename(columns={'dt_refer': 'date'}).iloc[:, 2:]
df_final_to_save.to_csv(save_path, index=False)

In [615]:
# TODO - Conferir detalhadamente
# display(df_final[df_final['dt_refer'] > '2021-01-01'].iloc[[0], 3:-1])
# display(df_final[df_final['dt_refer'] > '2020-01-01'].iloc[[0], 3:-1])
# display(df_final[df_final['dt_refer'] > '2019-01-01'].iloc[[0], 3:-1])

,caixa,deprec_e_amort,divida_bruta,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida,close_price,dividends,...,divida_liquida,LPA,VPA,P/L,P/EBITDA,P/VPA,DL/PL,DL/EBITDA,ROE,MARGEM_EBITDA
2257,64280000.0,58305000.0,392548000.0,49621000.0,6246000.0,311150000.0,1.304450e+10,272069000.0,28.91,0.0,...,328268000.0,0.478823,23.85297,60.377266,2.678687e-07,1.212008,1.055015,3.041603,0.020074,0.396686


,caixa,deprec_e_amort,divida_bruta,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida,close_price,dividends,...,divida_liquida,LPA,VPA,P/L,P/EBITDA,P/VPA,DL/PL,DL/EBITDA,ROE,MARGEM_EBITDA
2012,33294000.0,58502000.0,351161000.0,81701000.0,40970000.0,299137000.0,1.304450e+10,302245000.0,30.700001,0.0,...,317867000.0,3.140788,22.932046,9.774617,2.189682e-07,1.338738,1.062613,2.267191,0.136961,0.463872


,caixa,deprec_e_amort,divida_bruta,ebit,lucro_liquido,patrimonio_liquido,qtd_acoes,receita_liquida,close_price,dividends,...,divida_liquida,LPA,VPA,P/L,P/EBITDA,P/VPA,DL/PL,DL/EBITDA,ROE,MARGEM_EBITDA
1765,58052000.0,43646000.0,326876000.0,64876000.0,26698000.0,283543000.0,1.304450e+10,349836000.0,24.059999,0.0,...,268824000.0,2.046687,21.736599,11.755584,2.217062e-07,1.106889,0.948089,2.477138,0.094159,0.310208
